In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
import time as time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [2]:
# Instalation
s = Service((r'C:\Users\eli\Desktop\AllMyRepos\Web-Scraping-with-BeautifulSoup-and-Selenium\Project No.10\chromedriver.exe'))
chromeOptions = Options()
chromeOptions.headless = False
driver = webdriver.Chrome(service=s, options=chromeOptions)

In [3]:
# lists
titles_ = []
companies_ = []
ratings_ = []
locations_ = []
salaries_ = []

In [62]:
url = 'https://www.indeed.com/jobs?q=data+analyst&l=New+York%2C+NY&start=0&pp=gQAPAAABhIvz3qEAAAAB7kC5IwAnAQEBCFFbITFk9V4_HS90YSFZhUXFWI1BeZMmPL72AuEgB78FZWsAAAA&vjk=a44a90be68296825'
while True:
    # Get URL
    driver.get(url)
    time.sleep(3)
    
    # Soup Object
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    # Results
    results = soup.findAll('div', class_ = 'cardOutline')
    for result in results:
        try:  
            title = result.find('h2', class_ = 'jobTitle').text
            titles_.append(title)
        except:
            titles_.append(np.nan)

        try:
            company = result.find('span', class_ = 'companyName').text
            companies_.append(company)
        except:
            companies_.append(np.nan)

        try:
            rating = result.find('span', class_ = 'ratingNumber').text
            ratings_.append(rating)
        except:
            ratings_.append(np.nan)

        try:
            location = result.find('div', class_ = 'companyLocation').text
            locations_.append(location)
        except:
            locations_.append(location)

        salary = result.find(class_ = 'metadata salary-snippet-container')
        estimated_salary = result.find('div',class_ = 'metadata estimated-salary-container')
        if salary:
            salaries_.append(salary.text)
        elif estimated_salary:
            salaries_.append(estimated_salary.text)
        else:
            salaries_.append(np.nan)
    
    next_url1 = soup.find('a', attrs = {'aria-label' : 'Next Page'})
    next_url2 = soup.find('a', attrs = {'aria-label' : 'Next'})
    if next_url1:
        url = 'https://www.indeed.com/' + next_url1.get('href')
    elif next_url2:
        url = 'https://www.indeed.com/' + next_url2.get('href')
    else:
        break

In [63]:
df_job_posts = pd.DataFrame({ 'Title' : titles_,'Company': companies_,'Rating': ratings_, 
                           'Location' : locations_, 'Salary' : salaries_})

In [64]:
df_job_posts.drop_duplicates(keep = 'first', inplace = True, ignore_index = False)

In [65]:
df_job_posts.head(10)

,Title,Company,Rating,Location,Salary
0,Data Analyst,Fox Corporation,4.0,"Hybrid remote in New York, NY",Estimated $73.4K - $93K a year
1,Associate Analyst Reporting & Data Analytics,Madison Square Garden Entertainment,3.9,"New York, NY 10121 (Garment District area)",Estimated $64.7K - $81.9K a year
2,"Data Analyst, Product Operations",Riskified,NaN,"Hybrid remote in New York, NY 10001","$90,000 - $120,000 a year"
3,Part-time Data Analyst I (ONSITE: 18-19 hours ...,Mount Sinai,3.7,"New York, NY+6 locations","$58,661 - $74,250 a year"
4,Data Analyst,Dow Jones,3.9,"New York, NY 10176 (Murray Hill area)","$40,000 - $160,000 a year"
5,Part- Time Data Analyst,Mercy College,3.9,"Dobbs Ferry, NY 10522",$40 an hour
6,Data Analyst,S&P Global,3.9,"New York, NY+1 location","$74,067 - $128,019 a year"
7,Data Analyst,Tradeweb Markets LLC,3.7,"New York, NY 10001 (Chelsea area)",Estimated $91.9K - $116K a year
8,Data Analyst,UBS,3.8,"Weehawken, NJ 07086+2 locations",Estimated $78.4K - $99.3K a year
9,Data Analyst,Roles at YipitData,NaN,"Hybrid remote in New York, NY",Estimated $83.8K - $106K a year


In [66]:
df_job_posts['Rating'] = pd.to_numeric(df_job_posts['Rating'], errors = 'coerce')
df_job_posts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 958 entries, 0 to 1414
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Title     958 non-null    object 
 1   Company   958 non-null    object 
 2   Rating    760 non-null    float64
 3   Location  958 non-null    object 
 4   Salary    779 non-null    object 
dtypes: float64(1), object(4)
memory usage: 44.9+ KB


In [67]:
df_job_posts.to_excel('data_analyst_job_posts.xlsx', index = False)